#### Francesca Logiacco and Hilal Işık

# Final Project
This project is created and shared by the mentioned authors, 
with each joint author enjoying an equal interest in the undivided whole.


## Goal: 'Find best fitting activity for a Summer Day in Berlin-2022'

We want to give the best summer 2022(may-august) activity tipp according to the interests and max budget information of the user. For that purpose we are going to combine different sources:

1- Eventbrite
- Using the Webscraping method to download the free and paid event information

2- Gratis-in-Berlin
- Using the Webscraping method to download the event information

3- Parks and Lakes in Berlin
- Preparing csv files with the info from Berlin Bezirksamt and Visit Berlin websites. 

## This notebook contains the Webscraping of the paid events from Eventbrite

# Web Scraping EventBrite

In [59]:
# Import Lybraries and get the url

import sys
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import random 
import pickle

### PAID Events from Eventbrite in Berlin; 
### Month: MAY; 
### Category: MUSIC.
Link from eventbrite selecting the not-free events in May_Category: MUSIC (M)

In [60]:
#url = 'https://www.eventbrite.com/d/germany--berlin/music--events--this-month/paid-events/?page=1'

In [61]:
#Events_May_M = requests.get(url)

In [62]:
#Events_May_M = BeautifulSoup(Events_May_M.content, "html.parser")
#print(Events_May_M .prettify())

In [63]:
# links of all pages of the events of the current month 

iterations = range(1, 11)
for i in iterations:
    start_at= str(i)
    url = "https://www.eventbrite.com/d/germany--berlin/events--this-month/paid-events/?page=" + start_at
    print(url)

https://www.eventbrite.com/d/germany--berlin/events--this-month/paid-events/?page=1
https://www.eventbrite.com/d/germany--berlin/events--this-month/paid-events/?page=2
https://www.eventbrite.com/d/germany--berlin/events--this-month/paid-events/?page=3
https://www.eventbrite.com/d/germany--berlin/events--this-month/paid-events/?page=4
https://www.eventbrite.com/d/germany--berlin/events--this-month/paid-events/?page=5
https://www.eventbrite.com/d/germany--berlin/events--this-month/paid-events/?page=6
https://www.eventbrite.com/d/germany--berlin/events--this-month/paid-events/?page=7
https://www.eventbrite.com/d/germany--berlin/events--this-month/paid-events/?page=8
https://www.eventbrite.com/d/germany--berlin/events--this-month/paid-events/?page=9
https://www.eventbrite.com/d/germany--berlin/events--this-month/paid-events/?page=10


In [64]:
# A list with all the Event Names in May

Events_Names_M_M = []
Events_Dates_M_M = []
Events_Links_M_M = []
Events_Prices_M_M = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/music--events--this-month/paid-events/?page='
    url = url + str(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_M_M.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_M_M.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_M_M.append(link['href'])
        

    # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_M_M.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    


print(Events_Names_M_M)
print(Events_Dates_M_M)
print(Events_Links_M_M)
print(Events_Prices_M_M)

['kiezsalon opening w/ tarta relena and jason sharp', 'kiezsalon opening w/ richard skelton and booker stardrum', 'banana party', 'cdc festival boatride party', 'wahn: music, words, dance, performance', 'open mic night @ vagabund kesselhaus', 'wise by night', 'general levy', 'wise 2022  │  digital is (not) better!?', '[il]legal with nico moreno & basswell', 'gest invites 003: halogenix', "kilkelly's open sofa sessions (musicians/poets)", "michelle daly 'january skies' album launch", 'music @ lobe - record exchange', 'living room concert serie #2', 'strand liebe -  open air club festival w/ k-paul, mike la funk uvm.', 'tech it deep', 'beyond now pres. nakadia | bdme closing party', 'psychedelic vibe w/ major7 & x-noize', '28.05.2022 | epic fail party berlin i 300 kilo konfetti i und viel mehr <3', 'bar tausend x doormen on vacation', "olivia belli presents 'sol novo'", 'queer bedroom pop (feat. elio icaza + guests)', 'guča meets wacken „balkan metal party” special guest: balkan beat', '

In [65]:
print(len(Events_Names_M_M))
print(len(Events_Dates_M_M))
print(len(Events_Links_M_M))
print(len(Events_Prices_M_M))

41
41
41
41


In [66]:
# Make dictionary, keys that will become dataframe column names
Events_May_M_dict = {'Name':Events_Names_M_M, 'Date':Events_Dates_M_M, 'Price': Events_Prices_M_M, 'Links':Events_Links_M_M}

# Convert dictionary to Pandas dataframe
Events_May_M_df = pd.DataFrame(Events_May_M_dict)

# Set the index of the DataFrame from 1.
Events_May_M_df.index = Events_May_M_df.index + 1

# Data Cleaning (replace wrong values in 'Date' column). 
Events_May_M_df['Date'] = Events_May_M_df['Date'].str.replace(',','')
Events_May_M_df['Date'] = Events_May_M_df['Date'].astype(str)+'.05'
Events_May_M_df['Date'] = Events_May_M_df['Date'].str.replace('8:.05','Tomorrow')
Events_May_M_df['Date'] = Events_May_M_df['Date'].str.replace('7:.05','Tomorrow')
Events_May_M_df['Date'] = Events_May_M_df['Date'].str.replace('at.05','Tomorrow')

Events_May_M_df['Price'] = Events_May_M_df['Price'].str.replace('€','')
Events_May_M_df['Price'] = Events_May_M_df['Price'].str.replace('$','')
Events_May_M_df.drop(Events_May_M_df.loc[Events_May_M_df['Price']== 'Free'].index, inplace=True)
Events_May_M_df.drop(Events_May_M_df.loc[Events_May_M_df['Price']== 'Kostenlos'].index, inplace=True)

Events_May_M_df.columns = Events_May_M_df.columns.str.strip().str.lower()

Events_May_M_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\1482672356.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_May_M_df['Date'] = Events_May_M_df['Date'].str.replace('8:.05','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\1482672356.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_May_M_df['Date'] = Events_May_M_df['Date'].str.replace('7:.05','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\1482672356.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_May_M_df['Date'] = Events_May_M_df['Date'].str.replace('at.05','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\1482672356.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=Tr

,name,date,price,links
1,kiezsalon opening w/ tarta relena and jason sharp,20.05,"11,29",https://www.eventbrite.de/e/kiezsalon-opening-...
2,kiezsalon opening w/ richard skelton and booke...,21.05,"11,29",https://www.eventbrite.de/e/kiezsalon-opening-...
3,banana party,20.05,15,https://www.eventbrite.com/e/banana-party-tick...
4,cdc festival boatride party,28.05,43.80,https://www.eventbrite.de/e/cdc-festival-boatr...
5,"wahn: music, words, dance, performance",27.05,11.83,https://www.eventbrite.de/e/wahn-music-words-d...
7,wise by night,20.05,17.17,https://www.eventbrite.de/e/wise-by-night-tick...
8,general levy,21.05,7,https://www.eventbrite.de/e/general-levy-ticke...
9,wise 2022 │ digital is (not) better!?,Tomorrow,38.47,https://www.eventbrite.de/e/wise-2022-digital-...
10,[il]legal with nico moreno & basswell,27.05,"15,74",https://www.eventbrite.de/e/illegal-with-nico-...
11,gest invites 003: halogenix,20.05,16.09,https://www.eventbrite.de/e/gest-invites-003-h...


#### PAID Events from Eventbrite in Berlin; 
#### Month: June (J); 
#### Category: MUSIC.
Link from eventbrite selecting the paid events in June_Category: MUSIC (M)

In [67]:
url = 'https://www.eventbrite.com/d/germany--berlin/music--events--next-month/paid-events/?page=1'

In [68]:
# A list with all the Event Names in June

Events_Names_J_M = []
Events_Dates_J_M = []
Events_Links_J_M = []
Events_Prices_J_M = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/music--events--next-month/paid-events/?page='
    url = url + str(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]
    for name in names:
        Events_Names_J_M.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_J_M.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_J_M.append(link['href'])
    
    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_J_M.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    


print(len(Events_Names_J_M))
print(len(Events_Dates_J_M))
print(len(Events_Links_J_M))
print(len(Events_Prices_J_M))


26
26
26
26


In [69]:
# Make dictionary, keys that will become dataframe column names
Events_June_M_dict = {'Name':Events_Names_J_M, 'Date':Events_Dates_J_M, 'Price': Events_Prices_J_M, 'Links':Events_Links_J_M}

# Convert dictionary to Pandas dataframe
Events_June_M_df = pd.DataFrame(Events_June_M_dict)
Events_June_M_df.index = Events_June_M_df.index + 1
Events_June_M_df['Date'] = Events_June_M_df['Date'].str.replace(',','')
Events_June_M_df['Date'] = Events_June_M_df['Date'].astype(str)+'.06'
Events_June_M_df['Date'] = Events_June_M_df['Date'].str.replace('8:.06','Tomorrow')
Events_June_M_df['Date'] = Events_June_M_df['Date'].str.replace('at.06','Tomorrow')

Events_June_M_df['Price'] = Events_June_M_df['Price'].str.replace('€','')
Events_June_M_df['Price'] = Events_June_M_df['Price'].str.replace('$','')

Events_June_M_df.drop(Events_June_M_df.loc[Events_June_M_df['Price']== 'Free'].index, inplace=True)
Events_June_M_df.drop(Events_June_M_df.loc[Events_June_M_df['Price']== 'Kostenlos'].index, inplace=True)

Events_June_M_df.columns = Events_June_M_df.columns.str.strip().str.lower()
Events_June_M_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\461180559.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_June_M_df['Date'] = Events_June_M_df['Date'].str.replace('8:.06','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\461180559.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_June_M_df['Date'] = Events_June_M_df['Date'].str.replace('at.06','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\461180559.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_June_M_df['Price'] = Events_June_M_df['Price'].str.replace('$','')


,name,date,price,links
1,"kiezsalon w/ mario batkovic, hatis noit, park ...",18.06,"22,58",https://www.eventbrite.de/e/kiezsalon-w-mario-...
4,berlin carnival 2022 - 3 to 6 june | event + ...,3.06,251.70,https://www.eventbrite.co.uk/e/berlin-carnival...
5,50 jahre tipberlin – 20 years exberliner,18.06,19,https://www.eventbrite.de/e/50-jahre-tipberlin...
6,novafuture x .defaultbox open air,4.06,"15,99",https://www.eventbrite.com/e/novafuture-x-defa...
7,massar egbari (berlin),3.06,£27.54,https://www.eventbrite.co.uk/e/massar-egbari-b...
8,andrew bear live in maschinenhaus,18.06,17.17,https://www.eventbrite.com/e/andrew-bear-live-...
9,"queens alliance w/ sisters in dub (uk), roots ...",4.06,12,https://www.eventbrite.de/e/queens-alliance-w-...
10,infected feat. monty (f),18.06,"13,96",https://www.eventbrite.de/e/infected-feat-mont...
11,ignition - glam & glitter,2.06,13.96,https://www.eventbrite.co.uk/e/ignition-glam-g...
12,daniel romano's outfit,27.06,"22,49",https://www.eventbrite.de/e/daniel-romanos-out...


#### PAID Events from Eventbrite in Berlin; 
#### Month: July (JL); 
#### Category: MUSIC.
Link from eventbrite selecting the paid events in July_Category: MUSIC (M)

In [70]:
Events_Names_JL_M = []
Events_Dates_JL_M = []
Events_Links_JL_M = []
Events_Prices_JL_M = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/music--events/paid-events/?page='
    url = url + str(i) + '&start_date=2022-07-01&end_date=2022-07-31'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_JL_M.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_JL_M.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_JL_M.append(link['href'])

    # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_JL_M.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    

print(len(Events_Names_JL_M))
print(len(Events_Dates_JL_M))
print(len(Events_Links_JL_M))
print(len(Events_Prices_JL_M))


13
13
13
13


In [71]:
# Make dictionary, keys that will become dataframe column names
Events_JL_dict = {'Name':Events_Names_JL_M, 'Date':Events_Dates_JL_M, 'Price':Events_Prices_JL_M, 'Links':Events_Links_JL_M}

# Convert dictionary to Pandas dataframe
Events_July_M_df = pd.DataFrame(Events_JL_dict)
Events_July_M_df.index = Events_July_M_df.index + 1
Events_July_M_df['Date'] = Events_July_M_df['Date'].str.replace(',','')
Events_July_M_df['Date'] = Events_July_M_df['Date'].astype(str)+'.07'
Events_July_M_df['Date'] = Events_July_M_df['Date'].str.replace('8:.07','Tomorrow')
Events_July_M_df['Date'] = Events_July_M_df['Date'].str.replace('at.07','Tomorrow')

Events_July_M_df['Price'] = Events_July_M_df['Price'].str.replace('€','')
Events_July_M_df['Price'] = Events_July_M_df['Price'].str.replace('$','')

Events_July_M_df.drop(Events_July_M_df.loc[Events_July_M_df['Price']== 'Free'].index, inplace=True)
Events_July_M_df.drop(Events_July_M_df.loc[Events_July_M_df['Price']== 'Kostenlos'].index, inplace=True)

Events_July_M_df.columns = Events_July_M_df.columns.str.strip().str.lower()


Events_July_M_df


C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2124360007.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_July_M_df['Date'] = Events_July_M_df['Date'].str.replace('8:.07','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2124360007.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_July_M_df['Date'] = Events_July_M_df['Date'].str.replace('at.07','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2124360007.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_July_M_df['Price'] = Events_July_M_df['Price'].str.replace('$','')


,name,date,price,links
1,torture garden berlin midsummer ball,9.07,"22,05",https://www.eventbrite.co.uk/e/torture-garden-...
3,"zulu (usa), nervöus, eat my fear & deutsche la...",2.07,17.17,https://www.eventbrite.de/e/zulu-usa-nervous-e...
4,c4 - open air festival,9.07,"11,83",https://www.eventbrite.de/e/c4-open-air-festiv...
5,techno back to the root ́s,23.07,"23,15",https://www.eventbrite.de/e/techno-back-to-the...
6,ark white / prismala / rosa lee luna @ badehau...,28.07,12,https://www.eventbrite.com/e/ark-white-prismal...
7,dirty fences • berlin,29.07,"18,19",https://www.eventbrite.de/e/dirty-fences-berli...
8,kaurna cronin & band – european tour 2022,21.07,"15,99",https://www.eventbrite.de/e/kaurna-cronin-band...
9,30 waschhaus - das festival,23.07,30,https://www.eventbrite.de/e/30-waschhaus-das-f...
10,oriel quartett: in kollaboration 2 mit schulz ...,10.07,10,https://www.eventbrite.de/e/oriel-quartett-in-...


#### PAID Events from Eventbrite in Berlin; 
#### Month: August (A); 
#### Category: MUSIC.
Link from eventbrite selecting the not-free events in August_Category: MUSIC (M)

In [72]:
#url = 'https://www.eventbrite.com/d/germany--berlin/paid--music--events/?end_date=2022-08-31&page=1&start_date=2022-08-01'


In [73]:
Events_Names_A_M = []
Events_Dates_A_M = []
Events_Links_A_M = []
Events_Prices_A_M = []

for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/paid--music--events/?end_date=2022-08-31&page='
    url = url + str(i) + '&start_date=2022-08-01'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_A_M.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_A_M.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_A_M.append(link['href'])

        
    # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_A_M.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    

print(len(Events_Names_A_M))
print(len(Events_Dates_A_M))
print(len(Events_Links_A_M))
print(len(Events_Prices_A_M))


30
30
30
30


In [74]:
# Make dictionary, keys that will become dataframe column names
Events_A_dict = {'Name':Events_Names_A_M, 'Date':Events_Dates_A_M, 'Price': Events_Prices_A_M, 'Links':Events_Links_A_M}

# Convert dictionary to Pandas dataframe
Events_Aug_M_df = pd.DataFrame(Events_A_dict)

# Data Cleaning
Events_Aug_M_df.index = Events_Aug_M_df.index + 1
Events_Aug_M_df['Date'] = Events_Aug_M_df['Date'].str.replace(',','')
Events_Aug_M_df['Date'] = Events_Aug_M_df['Date'].astype(str)+'.08'
Events_Aug_M_df['Date'] = Events_Aug_M_df['Date'].str.replace('8:.08','Tomorrow')
Events_Aug_M_df['Date'] = Events_Aug_M_df['Date'].str.replace('at.08','Tomorrow')

Events_Aug_M_df['Price'] = Events_Aug_M_df['Price'].str.replace('€','')
Events_Aug_M_df['Price'] = Events_Aug_M_df['Price'].str.replace('$','')

Events_Aug_M_df.drop(Events_Aug_M_df.loc[Events_Aug_M_df['Price']== 'Free'].index, inplace=True)
Events_Aug_M_df.drop(Events_Aug_M_df.loc[Events_Aug_M_df['Price']== 'Kostenlos'].index, inplace=True)

Events_Aug_M_df.columns = Events_Aug_M_df.columns.str.strip().str.lower()


Events_Aug_M_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\407118269.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_Aug_M_df['Date'] = Events_Aug_M_df['Date'].str.replace('8:.08','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\407118269.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_Aug_M_df['Date'] = Events_Aug_M_df['Date'].str.replace('at.08','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\407118269.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_Aug_M_df['Price'] = Events_Aug_M_df['Price'].str.replace('$','')


,name,date,price,links
1,holi festival of colours berlin 2022,20.08,"10,36",https://www.eventbrite.de/e/holi-festival-of-c...
2,natiruts @ berlin,18.08,32.99,https://www.eventbrite.com/e/natiruts-berlin-t...
3,kiezsalon w/ microcorps and maja ratkje & more.,27.08,"22,58",https://www.eventbrite.de/e/kiezsalon-w-microc...
4,"schalala, das mitsingding",13.08,"13,96",https://www.eventbrite.de/e/schalala-das-mitsi...
5,festival son cubano - ecos de siboney,6.08,"17,17",https://www.eventbrite.de/e/festival-son-cuban...
6,lord zeppelin,5.08,"13,96",https://www.eventbrite.de/e/lord-zeppelin-tick...
7,"tixǝ - rock, blues + special guests",11.08,"10,65",https://www.eventbrite.de/e/tix-rock-blues-spe...
8,basta - eure liebsten lieder,19.08,"19,30",https://www.eventbrite.de/e/basta-eure-liebste...
9,eagel slam open air - best of poetry slam,12.08,"11,83",https://www.eventbrite.de/e/eagel-slam-open-ai...
10,farbgefühle festival berlin 2022,20.08,"10,36",https://www.eventbrite.de/e/farbgefuhle-festiv...


#### PAID Events from Eventbrite in Berlin; 
#### Month: May; 
#### Category: Community.


In [75]:
Events_Names_M_C = []
Events_Dates_M_C = []
Events_Links_M_C = []
Events_Prices_M_C = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/paid--community--events--this-month/?page='
    url = url + str(i) 
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_M_C.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_M_C.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_M_C.append(link['href'])

    # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_M_C.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    
        

print(len(Events_Names_M_C))
print(len(Events_Dates_M_C))
print(len(Events_Links_M_C))
print(len(Events_Prices_M_C))


19
19
19
19


In [76]:
# Make dictionary, keys that will become dataframe column names
Events_M_dict = {'Name':Events_Names_M_C, 'Date':Events_Dates_M_C, 'Price':Events_Prices_M_C, 'Links':Events_Links_M_C}

# Convert dictionary to Pandas dataframe
Events_M_C_df = pd.DataFrame(Events_M_dict)
Events_M_C_df.index = Events_M_C_df.index + 1
Events_M_C_df['Date'] = Events_M_C_df['Date'].str.replace(',','')
Events_M_C_df['Date'] = Events_M_C_df['Date'].astype(str)+'.05'
Events_M_C_df['Date'] = Events_M_C_df['Date'].str.replace('8:.05','Tomorrow')
Events_M_C_df['Date'] = Events_M_C_df['Date'].str.replace('at.05','Tomorrow')

Events_M_C_df['Price'] = Events_M_C_df['Price'].str.replace('€','')
Events_M_C_df['Price'] = Events_M_C_df['Price'].str.replace('$','')

Events_M_C_df.drop(Events_M_C_df.loc[Events_M_C_df['Price']== 'Free'].index, inplace=True)
Events_M_C_df.drop(Events_M_C_df.loc[Events_M_C_df['Price']== 'Kostenlos'].index, inplace=True)
Events_M_C_df.drop(Events_M_C_df.loc[Events_M_C_df['Price']== 'FreiwilligeSpende'].index, inplace=True)
Events_M_C_df.columns = Events_M_C_df.columns.str.strip().str.lower()


Events_M_C_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2104302540.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_M_C_df['Date'] = Events_M_C_df['Date'].str.replace('8:.05','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2104302540.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_M_C_df['Date'] = Events_M_C_df['Date'].str.replace('at.05','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2104302540.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_M_C_df['Price'] = Events_M_C_df['Price'].str.replace('$','')


,name,date,price,links
1,gypsy tales berlin,28.05,21.43,https://www.eventbrite.co.uk/e/gypsy-tales-ber...
2,silent.move walk'n'dance disco tour // berlin.,20.05,"11,90",https://www.eventbrite.de/e/silentmove-walknda...
3,sunday slips!,22.05,Donation,https://www.eventbrite.de/e/sunday-slips-ticke...
4,iwb may meet-up,25.05,11.83,https://www.eventbrite.de/e/iwb-may-meet-up-ti...
5,pachamama x christa kupfer,Tomorrow,6.51,https://www.eventbrite.com/e/pachamama-x-chris...
7,slam des westens #114 in the dark,25.05,"7,57",https://www.eventbrite.de/e/slam-des-westens-1...
8,noite cabo-verdiana/ stand up comedy com carlo...,28.05,25,https://www.eventbrite.de/e/bilhetes-noite-cab...
9,fundraising im chor | das 1x1 der antragsstellung,21.05,25,https://www.eventbrite.de/e/fundraising-im-cho...
10,"germania, berygtede bunkere og naziarkitektur",28.05,11.83,https://www.eventbrite.de/e/germania-berygtede...
11,dichterasse poetry slam,28.05,"7,57",https://www.eventbrite.de/e/dichterasse-poetry...


#### PAID Events from Eventbrite in Berlin; 
#### Month: June (J); 
#### Category: Community.

In [77]:
# A list with all the Event Names in June

Events_Names_J_C = []
Events_Dates_J_C = []
Events_Links_J_C = []
Events_Prices_J_C = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/community--events--next-month/paid-events/?page='
    url = url + str(i) 
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_J_C.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_J_C.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_J_C.append(link['href'])

        
        
    # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_J_C.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    
    

print(len(Events_Names_J_C))
print(len(Events_Dates_J_C))
print(len(Events_Links_J_C))
print(len(Events_Prices_J_C))


18
18
18
18


In [78]:
# Make dictionary, keys that will become dataframe column names
Events_J_dict = {'Name':Events_Names_J_C, 'Date':Events_Dates_J_C, 'Price': Events_Prices_J_C, 'Links':Events_Links_J_C}

# Convert dictionary to Pandas dataframe
Events_J_C_df = pd.DataFrame(Events_J_dict)
Events_J_C_df.index = Events_J_C_df.index + 1
Events_J_C_df['Date'] = Events_J_C_df['Date'].str.replace(',','')
Events_J_C_df['Date'] = Events_J_C_df['Date'].astype(str)+'.06'
Events_J_C_df['Date'] = Events_J_C_df['Date'].str.replace('8:.06','Tomorrow')
Events_J_C_df['Date'] = Events_J_C_df['Date'].str.replace('at.06','Tomorrow')


Events_J_C_df.drop(Events_J_C_df.loc[Events_J_C_df['Price']== '0€'].index, inplace=True)
Events_J_C_df['Price'] = Events_J_C_df['Price'].str.replace('€','')
Events_J_C_df['Price'] = Events_J_C_df['Price'].str.replace('$','')
Events_J_C_df.drop(Events_J_C_df.loc[Events_J_C_df['Price']== 'Free'].index, inplace=True)
Events_J_C_df.drop(Events_J_C_df.loc[Events_J_C_df['Price']== 'Kostenlos'].index, inplace=True)
Events_J_C_df.drop(Events_J_C_df.loc[Events_J_C_df['Price']== 'FreiwilligeSpende'].index, inplace=True)


Events_J_C_df.columns = Events_J_C_df.columns.str.strip().str.lower()

Events_J_C_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\3646760151.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_J_C_df['Date'] = Events_J_C_df['Date'].str.replace('8:.06','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\3646760151.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_J_C_df['Date'] = Events_J_C_df['Date'].str.replace('at.06','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\3646760151.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_J_C_df['Price'] = Events_J_C_df['Price'].str.replace('$','')


,name,date,price,links
1,sunday slips!,5.06,Donation,https://www.eventbrite.de/e/sunday-slips-ticke...
6,unshaved bear summer berlin edition 2022,25.06,"12,79",https://www.eventbrite.de/e/unshaved-bear-summ...
7,date deficient - vegan dating night,2.06,15,https://www.eventbrite.de/e/date-deficient-veg...
8,bearground - leider geil,5.06,"21,43",https://www.eventbrite.de/e/bearground-leider-...
9,silent.move walk'n'dance disco tour // berlin.,3.06,"11,90",https://www.eventbrite.de/e/silentmove-walknda...
10,juneteenth unity festival picnic in the park,19.06,15.25,https://www.eventbrite.de/e/juneteenth-unity-f...
11,kopftuchmädchen b-day event,4.06,"29,99",https://www.eventbrite.com/e/kopftuchmadchen-b...
12,liliana velásquez & chris davis,1.06,11.68,https://www.eventbrite.de/e/liliana-velasquez-...
13,dinner confidential,29.06,17.17,https://www.eventbrite.com/e/dinner-confidenti...
16,"hip berlin underground tour - street art, club...",2.06,81.09,https://www.eventbrite.de/e/hip-berlin-undergr...


#### PAID Events from Eventbrite in Berlin; 
#### Month: July (JL); 
#### Category: Community (C).

In [79]:
# A list with all the Event Names in July

Events_Names_JL_C = []
Events_Dates_JL_C = []
Events_Links_JL_C = []
Events_Prices_JL_C = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/community--events/paid-events/?page='
    url = url + str(i) + '&start_date=2022-07-01&end_date=2022-07-31'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_JL_C.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_JL_C.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_JL_C.append(link['href'])


        # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_JL_C.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    
        
print(len(Events_Names_JL_C))
print(len(Events_Dates_JL_C))
print(len(Events_Links_JL_C))
print(len(Events_Prices_JL_C))

5
5
5
5


In [80]:
# Make dictionary, keys that will become dataframe column names
Events_JL_dict = {'Name':Events_Names_JL_C, 'Date':Events_Dates_JL_C, 'Price': Events_Prices_JL_C, 'Links':Events_Links_JL_C}

# Convert dictionary to Pandas dataframe
Events_JL_C_df = pd.DataFrame(Events_JL_dict)
Events_JL_C_df.index = Events_JL_C_df.index + 1
Events_JL_C_df['Date'] = Events_JL_C_df['Date'].str.replace(',','')
Events_JL_C_df['Date'] = Events_JL_C_df['Date'].astype(str)+'.07'
Events_JL_C_df['Date'] = Events_JL_C_df['Date'].str.replace('8:.07','Tomorrow')
Events_JL_C_df['Date'] = Events_JL_C_df['Date'].str.replace('at.07','Tomorrow')

Events_JL_C_df.drop(Events_JL_C_df.loc[Events_JL_C_df['Price']== '0€'].index, inplace=True)
Events_JL_C_df['Price'] = Events_JL_C_df['Price'].str.replace('€','')
Events_JL_C_df['Price'] = Events_JL_C_df['Price'].str.replace('$','')
Events_JL_C_df.drop(Events_JL_C_df.loc[Events_JL_C_df['Price']== 'Free'].index, inplace=True)
Events_JL_C_df.drop(Events_JL_C_df.loc[Events_JL_C_df['Price']== 'Kostenlos'].index, inplace=True)
Events_JL_C_df.drop(Events_JL_C_df.loc[Events_JL_C_df['Price']== 'FreiwilligeSpende'].index, inplace=True)

Events_JL_C_df.columns = Events_JL_C_df.columns.str.strip().str.lower()


Events_JL_C_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\1607182174.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_JL_C_df['Date'] = Events_JL_C_df['Date'].str.replace('8:.07','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\1607182174.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_JL_C_df['Date'] = Events_JL_C_df['Date'].str.replace('at.07','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\1607182174.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_JL_C_df['Price'] = Events_JL_C_df['Price'].str.replace('$','')


,name,date,price,links
1,sunday slips!,3.07,Donation,https://www.eventbrite.de/e/sunday-slips-ticke...
2,silent.move walk'n'dance disco tour // berlin.,1.07,"11,90",https://www.eventbrite.de/e/silentmove-walknda...
3,the black wall street gala,9.07,"49,13",https://www.eventbrite.de/e/the-black-wall-str...
5,"hip berlin underground tour - street art, club...",1.07,81.09,https://www.eventbrite.de/e/hip-berlin-undergr...


#### PAID Events from Eventbrite in Berlin; 
#### Month: August (A); 
#### Category: Community.


In [81]:
# A list with all the Event Names in August

Events_Names_A_C = []
Events_Dates_A_C = []
Events_Links_A_C = []
Events_Prices_A_C = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/community--events/paid-events/?end_date=2022-08-31&page='
    url = url + str(i) + '&start_date=2022-08-01'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_A_C.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_A_C.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_A_C.append(link['href'])

        # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_A_C.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    

print(len(Events_Names_A_C))
print(len(Events_Dates_A_C))
print(len(Events_Links_A_C))
print(len(Events_Prices_A_C))


5
5
5
5


In [82]:
# Make dictionary, keys that will become dataframe column names
Events_A_dict = {'Name':Events_Names_A_C, 'Date':Events_Dates_A_C, 'Price':Events_Prices_A_C, 'Links':Events_Links_A_C}

# Convert dictionary to Pandas dataframe
Events_A_C_df = pd.DataFrame(Events_A_dict)
Events_A_C_df.index = Events_A_C_df.index + 1
Events_A_C_df['Date'] = Events_A_C_df['Date'].str.replace(',','')
Events_A_C_df['Date'] = Events_A_C_df['Date'].astype(str)+'.08'
Events_A_C_df['Date'] = Events_A_C_df['Date'].str.replace('8:.08','Tomorrow')
Events_A_C_df['Date'] = Events_A_C_df['Date'].str.replace('at.08','Tomorrow')

Events_A_C_df.drop(Events_A_C_df.loc[Events_A_C_df['Price']== '0€'].index, inplace=True)
Events_A_C_df['Price'] = Events_A_C_df['Price'].str.replace('€','')
Events_A_C_df['Price'] = Events_A_C_df['Price'].str.replace('$','')
Events_A_C_df.drop(Events_A_C_df.loc[Events_A_C_df['Price']== 'Free'].index, inplace=True)
Events_A_C_df.drop(Events_A_C_df.loc[Events_A_C_df['Price']== 'Kostenlos'].index, inplace=True)
Events_A_C_df.drop(Events_A_C_df.loc[Events_A_C_df['Price']== 'FreiwilligeSpende'].index, inplace=True)


Events_A_C_df.columns = Events_A_C_df.columns.str.strip().str.lower()


Events_A_C_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\618986615.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_A_C_df['Date'] = Events_A_C_df['Date'].str.replace('8:.08','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\618986615.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_A_C_df['Date'] = Events_A_C_df['Date'].str.replace('at.08','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\618986615.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_A_C_df['Price'] = Events_A_C_df['Price'].str.replace('$','')


,name,date,price,links
1,sunday slips!,7.08,Donation,https://www.eventbrite.de/e/sunday-slips-ticke...
2,silent.move walk'n'dance disco tour // berlin.,5.08,"11,90",https://www.eventbrite.de/e/silentmove-walknda...
3,whipped cream - untertage berlin,27.08,19,https://www.eventbrite.de/e/whipped-cream-unte...
5,"hip berlin underground tour - street art, club...",4.08,81.09,https://www.eventbrite.de/e/hip-berlin-undergr...


#### PAID Events from Eventbrite in Berlin; 
#### Month: May (M); 
#### Category: Cinema (Cin).


In [83]:
# A list with all the Event Names in May

Events_Names_M_Cin = []
Events_Dates_M_Cin = []
Events_Links_M_Cin = []
Events_Prices_M_Cin = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/film-and-media--events--this-month/paid-events/?page='
    url = url + str(i) 
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_M_Cin.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_M_Cin.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_M_Cin.append(link['href'])

         # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_M_Cin.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    

print(len(Events_Names_M_Cin))
print(len(Events_Dates_M_Cin))
print(len(Events_Links_M_Cin))
print(len(Events_Prices_M_Cin))


28
28
28
28


In [84]:
# Make dictionary, keys that will become dataframe column names
Events_M_dict = {'Name':Events_Names_M_Cin, 'Date':Events_Dates_M_Cin, 'Price':Events_Prices_M_Cin, 'Links':Events_Links_M_Cin}

# Convert dictionary to Pandas dataframe
Events_M_Cin_df = pd.DataFrame(Events_M_dict)
Events_M_Cin_df.index = Events_M_Cin_df.index + 1
Events_M_Cin_df['Date'] = Events_M_Cin_df['Date'].str.replace(',','')
Events_M_Cin_df['Date'] = Events_M_Cin_df['Date'].astype(str)+'.05'
Events_M_Cin_df['Date'] = Events_M_Cin_df['Date'].str.replace('8:.05','Tomorrow')
Events_M_Cin_df['Date'] = Events_M_Cin_df['Date'].str.replace('at.05','Tomorrow')

Events_M_Cin_df.drop(Events_M_Cin_df.loc[Events_M_Cin_df['Price']== '0€'].index, inplace=True)
Events_M_Cin_df['Price'] = Events_M_Cin_df['Price'].str.replace('€','')
Events_M_Cin_df['Price'] = Events_M_Cin_df['Price'].str.replace('$','')
Events_M_Cin_df.drop(Events_M_Cin_df.loc[Events_M_Cin_df['Price']== 'Free'].index, inplace=True)
Events_M_Cin_df.drop(Events_M_Cin_df.loc[Events_M_Cin_df['Price']== 'Kostenlos'].index, inplace=True)
Events_M_Cin_df.drop(Events_M_Cin_df.loc[Events_M_Cin_df['Price']== 'FreiwilligeSpende'].index, inplace=True)

Events_M_Cin_df.columns = Events_M_Cin_df.columns.str.strip().str.lower()



Events_M_Cin_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2403355244.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_M_Cin_df['Date'] = Events_M_Cin_df['Date'].str.replace('8:.05','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2403355244.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_M_Cin_df['Date'] = Events_M_Cin_df['Date'].str.replace('at.05','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2403355244.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_M_Cin_df['Price'] = Events_M_Cin_df['Price'].str.replace('$','')


,name,date,price,links
1,english comedy berlin with pizza and shots,20.05,"22,51",https://www.eventbrite.de/e/english-comedy-ber...
2,english stand up comedy - culture shock specia...,21.05,12.54,https://www.eventbrite.co.uk/e/english-stand-u...
5,that's what she said - womxn comedy night,21.05,16,https://www.eventbrite.de/e/thats-what-she-sai...
7,new york comedy club - berlin: main show,23.05,2.14,https://www.eventbrite.com/e/new-york-comedy-c...
9,comedy roulette | the ultimate comedy game show,20.05,13,https://www.eventbrite.de/e/comedy-roulette-th...
11,laughing spree: english comedy on a boat (free...,24.05,2.13,https://www.eventbrite.de/e/laughing-spree-eng...
13,expats in berlin #35 - english comedy show,26.05,8.55,https://www.eventbrite.co.uk/e/expats-in-berli...
14,new york comedy club - berlin: late show - dar...,23.05,2.14,https://www.eventbrite.com/e/new-york-comedy-c...
18,high bars & low morals. english comedy special...,22.05,10,https://www.eventbrite.com/e/high-bars-low-mor...
19,"hook, value and emotions. video content in 202...",25.05,0,https://www.eventbrite.de/e/hook-value-and-emo...


#### PAID Events from Eventbrite in Berlin; 
#### Month: June (J); 
#### Category: Cinema.


In [85]:
# A list with all the Event Names in June

Events_Names_J_Cin = []
Events_Dates_J_Cin = []
Events_Links_J_Cin = []
Events_Prices_J_Cin = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/film-and-media--events--next-month/paid-events/?page='
    url = url + str(i) 
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_J_Cin.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_J_Cin.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_J_Cin.append(link['href'])

        
        # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_J_Cin.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    

print(len(Events_Names_J_Cin))
print(len(Events_Dates_J_Cin))
print(len(Events_Links_J_Cin))
print(len(Events_Prices_J_Cin))


21
21
21
21


In [86]:
# Make dictionary, keys that will become dataframe column names
Events_J_dict = {'Name':Events_Links_J_Cin, 'Date':Events_Dates_J_Cin, 'Price':Events_Prices_J_Cin, 'Links':Events_Links_J_Cin}

# Convert dictionary to Pandas dataframe
Events_J_Cin_df = pd.DataFrame(Events_J_dict)
Events_J_Cin_df.index = Events_J_Cin_df.index + 1
Events_J_Cin_df['Date'] = Events_J_Cin_df['Date'].str.replace(',','')
Events_J_Cin_df['Date'] = Events_J_Cin_df['Date'].astype(str)+'.06'
Events_J_Cin_df['Date'] = Events_J_Cin_df['Date'].str.replace('8:.06','Tomorrow')
Events_J_Cin_df['Date'] = Events_J_Cin_df['Date'].str.replace('at.06','Tomorrow')

Events_J_Cin_df.drop(Events_J_Cin_df.loc[Events_J_Cin_df['Price']== '0€'].index, inplace=True)
Events_J_Cin_df['Price'] = Events_J_Cin_df['Price'].str.replace('€','')
Events_J_Cin_df['Price'] = Events_J_Cin_df['Price'].str.replace('$','')
Events_J_Cin_df.drop(Events_J_Cin_df.loc[Events_J_Cin_df['Price']== 'Free'].index, inplace=True)
Events_J_Cin_df.drop(Events_J_Cin_df.loc[Events_J_Cin_df['Price']== 'Kostenlos'].index, inplace=True)
Events_J_Cin_df.drop(Events_J_Cin_df.loc[Events_J_Cin_df['Price']== 'FreiwilligeSpende'].index, inplace=True)

Events_J_Cin_df.columns = Events_J_Cin_df.columns.str.strip().str.lower()


Events_J_Cin_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\876609229.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_J_Cin_df['Date'] = Events_J_Cin_df['Date'].str.replace('8:.06','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\876609229.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_J_Cin_df['Date'] = Events_J_Cin_df['Date'].str.replace('at.06','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\876609229.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_J_Cin_df['Price'] = Events_J_Cin_df['Price'].str.replace('$','')


,name,date,price,links
1,https://www.eventbrite.de/e/english-comedy-ber...,3.06,"22,51",https://www.eventbrite.de/e/english-comedy-ber...
2,https://www.eventbrite.co.uk/e/english-stand-u...,18.06,11.77,https://www.eventbrite.co.uk/e/english-stand-u...
3,https://www.eventbrite.de/e/berlin-music-video...,8.06,22.49,https://www.eventbrite.de/e/berlin-music-video...
4,https://www.eventbrite.de/e/berlin-freak-burle...,10.06,30,https://www.eventbrite.de/e/berlin-freak-burle...
5,https://www.eventbrite.de/e/yaaas-queen-womxn-...,8.06,14,https://www.eventbrite.de/e/yaaas-queen-womxn-...
7,https://www.eventbrite.com/e/new-york-comedy-c...,27.06,2.14,https://www.eventbrite.com/e/new-york-comedy-c...
8,https://www.eventbrite.co.uk/e/english-stand-u...,4.06,12.54,https://www.eventbrite.co.uk/e/english-stand-u...
9,https://www.eventbrite.de/e/thats-what-she-sai...,25.06,16,https://www.eventbrite.de/e/thats-what-she-sai...
10,https://www.eventbrite.com/e/new-york-comedy-c...,6.06,2.14,https://www.eventbrite.com/e/new-york-comedy-c...
12,https://www.eventbrite.de/e/laughing-spree-eng...,5.06,2.26,https://www.eventbrite.de/e/laughing-spree-eng...


#### PAID Events from Eventbrite in Berlin; 
#### Month: July (JL); 
#### Category: Cinema.


In [87]:
# A list with all the Event Names in July

Events_Names_JL_Cin = []
Events_Dates_JL_Cin = []
Events_Links_JL_Cin = []
Events_Prices_JL_Cin = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/film-and-media--events/paid-events/?page='
    url = url + str(i) + '&start_date=2022-07-01&end_date=2022-07-31'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_JL_Cin.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_JL_Cin.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_JL_Cin.append(link['href'])

        
        
        # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_JL_Cin.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    

    
print(len(Events_Names_JL_Cin))
print(len(Events_Dates_JL_Cin))
print(len(Events_Links_JL_Cin))
print(len(Events_Prices_JL_Cin))


5
5
5
5


In [88]:
# Make dictionary, keys that will become dataframe column names
Events_JL_dict = {'Name':Events_Names_JL_Cin, 'Date':Events_Dates_JL_Cin, 'Price': Events_Prices_JL_Cin, 'Links':Events_Links_JL_Cin}

# Convert dictionary to Pandas dataframe
Events_JL_Cin_df = pd.DataFrame(Events_JL_dict)
Events_JL_Cin_df.index = Events_JL_Cin_df.index + 1
Events_JL_Cin_df['Date'] = Events_JL_Cin_df['Date'].str.replace(',','')
Events_JL_Cin_df['Date'] = Events_JL_Cin_df['Date'].astype(str)+'.07'
Events_JL_Cin_df['Date'] = Events_JL_Cin_df['Date'].str.replace('8:.07','Tomorrow')
Events_JL_Cin_df['Date'] = Events_JL_Cin_df['Date'].str.replace('at.07','Tomorrow')

Events_JL_Cin_df.drop(Events_JL_Cin_df.loc[Events_JL_Cin_df['Price']== '0€'].index, inplace=True)
Events_JL_Cin_df['Price'] = Events_JL_Cin_df['Price'].str.replace('€','')
Events_JL_Cin_df['Price'] = Events_JL_Cin_df['Price'].str.replace('$','')
Events_JL_Cin_df.drop(Events_JL_Cin_df.loc[Events_JL_Cin_df['Price']== 'Free'].index, inplace=True)
Events_JL_Cin_df.drop(Events_JL_Cin_df.loc[Events_JL_Cin_df['Price']== 'Kostenlos'].index, inplace=True)
Events_JL_Cin_df.drop(Events_JL_Cin_df.loc[Events_JL_Cin_df['Price']== 'FreiwilligeSpende'].index, inplace=True)

Events_JL_Cin_df.columns = Events_JL_Cin_df.columns.str.strip().str.lower()


Events_JL_Cin_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\3618062270.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_JL_Cin_df['Date'] = Events_JL_Cin_df['Date'].str.replace('8:.07','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\3618062270.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_JL_Cin_df['Date'] = Events_JL_Cin_df['Date'].str.replace('at.07','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\3618062270.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_JL_Cin_df['Price'] = Events_JL_Cin_df['Price'].str.replace('$','')


,name,date,price,links
1,english comedy berlin with pizza and shots,1.07,"22,51",https://www.eventbrite.de/e/english-comedy-ber...
2,yaaas queen! | womxn & lgbtq+ comedy night,13.07,14,https://www.eventbrite.de/e/yaaas-queen-womxn-...
3,the witching hour,2.07,Donation,https://www.eventbrite.com/e/the-witching-hour...
5,movie campout weekend,1.07,10,https://www.eventbrite.com/e/movie-campout-wee...


#### PAID Events from Eventbrite in Berlin; 
#### Month: August (A); 
#### Category: Cinema.


In [89]:
Events_Names_A_Cin = []
Events_Dates_A_Cin = []
Events_Links_A_Cin = []
Events_Prices_A_Cin = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/film-and-media--events/paid-events/?end_date=2022-08-31&page='
    url = url + str(i) + '&start_date=2022-08-01'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_A_Cin.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_A_Cin.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_A_Cin.append(link['href'])

        
        # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_A_Cin.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    

print(len(Events_Names_A_Cin))
print(len(Events_Dates_A_Cin))
print(len(Events_Links_A_Cin))
print(len(Events_Prices_A_Cin))


1
1
1
1


In [90]:
# Make dictionary, keys that will become dataframe column names
Events_A_dict = {'Name':Events_Names_A_Cin, 'Date':Events_Dates_A_Cin, 'Price':Events_Prices_A_Cin, 'Links':Events_Links_A_Cin}

# Convert dictionary to Pandas dataframe
Events_A_Cin_df = pd.DataFrame(Events_A_dict)
Events_A_Cin_df.index = Events_A_Cin_df.index + 1

Events_A_Cin_df['Date'] = Events_A_Cin_df['Date'].str.replace(',','')
Events_A_Cin_df['Date'] = Events_A_Cin_df['Date'].astype(str)+'.08'
Events_A_Cin_df['Date'] = Events_A_Cin_df['Date'].str.replace('8:.08','Tomorrow')
Events_A_Cin_df['Date'] = Events_A_Cin_df['Date'].str.replace('at.08','Tomorrow')

Events_A_Cin_df.drop(Events_A_Cin_df.loc[Events_A_Cin_df['Price']== '0€'].index, inplace=True)
Events_A_Cin_df['Price'] = Events_A_Cin_df['Price'].str.replace('€','')
Events_A_Cin_df['Price'] = Events_A_Cin_df['Price'].str.replace('$','')
Events_A_Cin_df.drop(Events_A_Cin_df.loc[Events_A_Cin_df['Price']== 'Free'].index, inplace=True)
Events_A_Cin_df.drop(Events_A_Cin_df.loc[Events_A_Cin_df['Price']== 'Kostenlos'].index, inplace=True)
Events_A_Cin_df.drop(Events_A_Cin_df.loc[Events_A_Cin_df['Price']== 'FreiwilligeSpende'].index, inplace=True)

Events_A_Cin_df.columns = Events_A_Cin_df.columns.str.strip().str.lower()


Events_A_Cin_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\386556269.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_A_Cin_df['Date'] = Events_A_Cin_df['Date'].str.replace('8:.08','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\386556269.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_A_Cin_df['Date'] = Events_A_Cin_df['Date'].str.replace('at.08','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\386556269.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_A_Cin_df['Price'] = Events_A_Cin_df['Price'].str.replace('$','')


,name,date,price,links
1,the witching hour,6.08,Donation,https://www.eventbrite.com/e/the-witching-hour...


#### PAID Events from Eventbrite in Berlin; 
#### Month: May (M); 
#### Category: Science (S).

In [91]:
# A list with all the Event Names in May

Events_Names_M_S = []
Events_Dates_M_S = []
Events_Links_M_S = []
Events_Prices_M_S = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/science-and-tech--events--this-month/paid-events/?page='
    url = url + str(i) 
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_M_S.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_M_S.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_M_S.append(link['href'])

        
        # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_M_S.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    

print(len(Events_Names_M_S))
print(len(Events_Dates_M_S))
print(len(Events_Links_M_S))
print(len(Events_Prices_M_S))

1
1
1
1


In [92]:
# Make dictionary, keys that will become dataframe column names
Events_M_dict = {'Name':Events_Names_M_S, 'Date':Events_Dates_M_S, 'Price': Events_Prices_M_S,'Links':Events_Links_M_S}

# Convert dictionary to Pandas dataframe
Events_M_S_df = pd.DataFrame(Events_M_dict)
Events_M_S_df.index = Events_M_S_df.index + 1
Events_M_S_df['Date'] = Events_M_S_df['Date'].str.replace(',','')
Events_M_S_df['Date'] = Events_M_S_df['Date'].astype(str)+'.05'
Events_M_S_df['Date'] = Events_M_S_df['Date'].str.replace('8:.05','Tomorrow')
Events_M_S_df['Date'] = Events_M_S_df['Date'].str.replace('at.05','Tomorrow')

Events_M_S_df.drop(Events_M_S_df.loc[Events_M_S_df['Price']== '0€'].index, inplace=True)
Events_M_S_df['Price'] = Events_M_S_df['Price'].str.replace('€','')
Events_M_S_df['Price'] = Events_M_S_df['Price'].str.replace('$','')
Events_M_S_df.drop(Events_M_S_df.loc[Events_M_S_df['Price']== 'Free'].index, inplace=True)
Events_M_S_df.drop(Events_M_S_df.loc[Events_M_S_df['Price']== 'Kostenlos'].index, inplace=True)
Events_M_S_df.drop(Events_M_S_df.loc[Events_M_S_df['Price']== 'FreiwilligeSpende'].index, inplace=True)

Events_M_S_df.columns = Events_M_S_df.columns.str.strip().str.lower()


Events_M_S_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\751792702.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_M_S_df['Date'] = Events_M_S_df['Date'].str.replace('8:.05','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\751792702.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_M_S_df['Date'] = Events_M_S_df['Date'].str.replace('at.05','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\751792702.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_M_S_df['Price'] = Events_M_S_df['Price'].str.replace('$','')


,name,date,price,links


#### PAID Events from Eventbrite in Berlin; 
#### Month: June (J); 
#### Category: Science (S).

In [93]:
# A list with all the Event Names in June

Events_Names_J_S = []
Events_Dates_J_S = []
Events_Links_J_S = []
Events_Prices_J_S = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/science-and-tech--events--next-month/paid-events/?page='
    url = url + str(i) 
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_J_S.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_J_S.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_J_S.append(link['href'])

        # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_J_S.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    


print(len(Events_Names_J_S))
print(len(Events_Dates_J_S))
print(len(Events_Links_J_S))
print(len(Events_Prices_J_S))


6
6
6
6


In [94]:
# Make dictionary, keys that will become dataframe column names
Events_J_dict = {'Name':Events_Names_J_S, 'Date':Events_Dates_J_S, 'Price':Events_Prices_J_S, 'Links':Events_Links_J_S}

# Convert dictionary to Pandas dataframe
Events_J_S_df = pd.DataFrame(Events_J_dict)
Events_J_S_df.index = Events_J_S_df.index + 1
Events_J_S_df['Date'] = Events_J_S_df['Date'].str.replace(',','')
Events_J_S_df['Date'] = Events_J_S_df['Date'].astype(str)+'.06'
Events_J_S_df['Date'] = Events_J_S_df['Date'].str.replace('8:.06','Tomorrow')
Events_J_S_df['Date'] = Events_J_S_df['Date'].str.replace('at.06','Tomorrow')

Events_J_S_df.drop(Events_J_S_df.loc[Events_J_S_df['Price']== '0€'].index, inplace=True)
Events_J_S_df['Price'] = Events_J_S_df['Price'].str.replace('€','')
Events_J_S_df['Price'] = Events_J_S_df['Price'].str.replace('$','')
Events_J_S_df.drop(Events_J_S_df.loc[Events_J_S_df['Price']== 'Free'].index, inplace=True)
Events_J_S_df.drop(Events_J_S_df.loc[Events_J_S_df['Price']== 'Kostenlos'].index, inplace=True)
Events_J_S_df.drop(Events_J_S_df.loc[Events_J_S_df['Price']== 'FreiwilligeSpende'].index, inplace=True)
Events_J_S_df.drop(Events_J_S_df.loc[Events_J_S_df['Price']== '0'].index, inplace=True)

Events_J_S_df.columns = Events_J_S_df.columns.str.strip().str.lower()


Events_J_S_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\1647119662.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_J_S_df['Date'] = Events_J_S_df['Date'].str.replace('8:.06','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\1647119662.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_J_S_df['Date'] = Events_J_S_df['Date'].str.replace('at.06','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\1647119662.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_J_S_df['Price'] = Events_J_S_df['Price'].str.replace('$','')


,name,date,price,links
1,:clojured berlin conference 2022,11.06,79,https://www.eventbrite.de/e/clojured-berlin-co...
2,openinfra summit berlin 2022,7.06,"1,199",https://www.eventbrite.com/e/openinfra-summit-...


#### PAID Events from Eventbrite in Berlin; 
#### Month: July (JL); 
#### Category: Science (S).


In [95]:
# A list with all the Event Names in July

Events_Names_JL_S = []
Events_Dates_JL_S = []
Events_Links_JL_S = []
Events_Prices_JL_S= []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/science-and-tech--events/paid-events/?page='
    url = url + str(i) + '&start_date=2022-07-01&end_date=2022-07-31'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_JL_S.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_JL_S.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_JL_S.append(link['href'])

        
        
        # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_JL_S.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    


print(len(Events_Names_JL_S))
print(len(Events_Dates_JL_S))
print(len(Events_Links_JL_S))
print(len(Events_Prices_JL_S))


1
1
1
1


In [96]:
# Make dictionary, keys that will become dataframe column names
Events_JL_dict = {'Name':Events_Names_JL_S, 'Date':Events_Dates_JL_S, 'Price': Events_Prices_JL_S, 'Links':Events_Links_JL_S}

# Convert dictionary to Pandas dataframe
Events_JL_S_df = pd.DataFrame(Events_JL_dict)
Events_JL_S_df.index = Events_JL_S_df.index + 1
Events_JL_S_df['Date'] = Events_JL_S_df['Date'].str.replace(',','')
Events_JL_S_df['Date'] = Events_JL_S_df['Date'].astype(str)+'.07'
Events_JL_S_df['Date'] = Events_JL_S_df['Date'].str.replace('8:.07','Tomorrow')
Events_JL_S_df['Date'] = Events_JL_S_df['Date'].str.replace('at.07','Tomorrow')

Events_JL_S_df.drop(Events_JL_S_df.loc[Events_JL_S_df['Price']== '0€'].index, inplace=True)
Events_JL_S_df['Price'] = Events_JL_S_df['Price'].str.replace('€','')
Events_JL_S_df['Price'] = Events_JL_S_df['Price'].str.replace('$','')
Events_JL_S_df.drop(Events_JL_S_df.loc[Events_JL_S_df['Price']== 'Free'].index, inplace=True)
Events_JL_S_df.drop(Events_JL_S_df.loc[Events_JL_S_df['Price']== 'Kostenlos'].index, inplace=True)
Events_JL_S_df.drop(Events_JL_S_df.loc[Events_JL_S_df['Price']== 'FreiwilligeSpende'].index, inplace=True)
Events_JL_S_df.drop(Events_JL_S_df.loc[Events_JL_S_df['Price']== '0'].index, inplace=True)
Events_JL_S_df.columns = Events_JL_S_df.columns.str.strip().str.lower()


Events_JL_S_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\1055674243.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_JL_S_df['Date'] = Events_JL_S_df['Date'].str.replace('8:.07','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\1055674243.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_JL_S_df['Date'] = Events_JL_S_df['Date'].str.replace('at.07','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\1055674243.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_JL_S_df['Price'] = Events_JL_S_df['Price'].str.replace('$','')


,name,date,price,links
1,gophercon europe 2022 - online and in-person,28.07,126.72,https://www.eventbrite.com/e/gophercon-europe-...


#### PAID Events from Eventbrite in Berlin; 
#### Month: August (A); 
#### Category: Science (S).

In [97]:
# A list with all the Event Names in August

Events_Names_A_S = []
Events_Dates_A_S = []
Events_Links_A_S = []
Events_Prices_A_S = []

for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/science-and-tech--events/paid-events/?end_date=2022-08-31&page='
    url = url + str(i) + '&start_date=2022-08-01'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_A_S.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_A_S.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_A_S.append(link['href'])

         
        # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_A_S.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    



print(len(Events_Names_A_S))
print(len(Events_Dates_A_S))
print(len(Events_Links_A_S))
print(len(Events_Prices_A_S))


1
1
1
1


In [98]:
# Make dictionary, keys that will become dataframe column names
Events_A_dict = {'Name':Events_Names_A_S, 'Date':Events_Dates_A_S, 'Price':Events_Prices_A_S, 'Links':Events_Links_A_S}

# Convert dictionary to Pandas dataframe
Events_A_S_df = pd.DataFrame(Events_A_dict)
Events_A_S_df.index = Events_A_S_df.index + 1
Events_A_S_df['Date'] = Events_A_S_df['Date'].str.replace(',','')
Events_A_S_df['Date'] = Events_A_S_df['Date'].astype(str)+'.08'
Events_A_S_df['Date'] = Events_A_S_df['Date'].str.replace('8:.08','Tomorrow')
Events_A_S_df['Date'] = Events_A_S_df['Date'].str.replace('at.08','Tomorrow')

Events_A_S_df.drop(Events_A_S_df.loc[Events_A_S_df['Price']== '0€'].index, inplace=True)
Events_A_S_df['Price'] = Events_A_S_df['Price'].str.replace('€','')
Events_A_S_df['Price'] = Events_A_S_df['Price'].str.replace('$','')
Events_A_S_df.drop(Events_A_S_df.loc[Events_A_S_df['Price']== 'Free'].index, inplace=True)
Events_A_S_df.drop(Events_A_S_df.loc[Events_A_S_df['Price']== 'Kostenlos'].index, inplace=True)
Events_A_S_df.drop(Events_A_S_df.loc[Events_A_S_df['Price']== 'FreiwilligeSpende'].index, inplace=True)
Events_A_S_df.drop(Events_A_S_df.loc[Events_A_S_df['Price']== '0'].index, inplace=True)

Events_A_S_df.columns = Events_A_S_df.columns.str.strip().str.lower()


Events_A_S_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2646833629.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_A_S_df['Date'] = Events_A_S_df['Date'].str.replace('8:.08','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2646833629.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_A_S_df['Date'] = Events_A_S_df['Date'].str.replace('at.08','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2646833629.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_A_S_df['Price'] = Events_A_S_df['Price'].str.replace('$','')


,name,date,price,links
1,4th edition - sx tech eu conference - berlin -...,31.08,74.70,https://www.eventbrite.ie/e/4th-edition-sx-tec...


#### PAID Events from Eventbrite in Berlin; 
#### Month: May (M); 
#### Category: Sport (Sp).

In [99]:
# A list with all the Event Names in May

Events_Names_M_Sp = []
Events_Dates_M_Sp = []
Events_Links_M_Sp = []
Events_Prices_M_Sp = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/sports-and-fitness--events--this-month/paid-events/?page='
    url = url + str(i) 
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_M_Sp.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_M_Sp.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_M_Sp.append(link['href'])

         
        # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_M_Sp.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    


print(len(Events_Names_M_Sp))
print(len(Events_Dates_M_Sp))
print(len(Events_Links_M_Sp))
print(len(Events_Prices_M_Sp))


8
8
8
8


In [100]:
# Make dictionary, keys that will become dataframe column names
Events_M_dict = {'Name':Events_Names_M_Sp, 'Date':Events_Dates_M_Sp, 'Price':Events_Prices_M_Sp, 'Links':Events_Links_M_Sp}

# Convert dictionary to Pandas dataframe
Events_M_Sp_df = pd.DataFrame(Events_M_dict)
Events_M_Sp_df.index = Events_M_Sp_df.index + 1
Events_M_Sp_df['Date'] = Events_M_Sp_df['Date'].str.replace(',','')
Events_M_Sp_df['Date'] = Events_M_Sp_df['Date'].astype(str)+'.05'
Events_M_Sp_df['Date'] = Events_M_Sp_df['Date'].str.replace('8:.05','Tomorrow')
Events_M_Sp_df['Date'] = Events_M_Sp_df['Date'].str.replace('at.05','Tomorrow')

Events_M_Sp_df.drop(Events_M_Sp_df.loc[Events_M_Sp_df['Price']== '0€'].index, inplace=True)
Events_M_Sp_df['Price'] = Events_M_Sp_df['Price'].str.replace('€','')
Events_M_Sp_df['Price'] = Events_M_Sp_df['Price'].str.replace('$','')
Events_M_Sp_df.drop(Events_M_Sp_df.loc[Events_M_Sp_df['Price']== 'Free'].index, inplace=True)
Events_M_Sp_df.drop(Events_M_Sp_df.loc[Events_M_Sp_df['Price']== 'Kostenlos'].index, inplace=True)
Events_M_Sp_df.drop(Events_M_Sp_df.loc[Events_M_Sp_df['Price']== 'FreiwilligeSpende'].index, inplace=True)
Events_M_Sp_df.drop(Events_M_Sp_df.loc[Events_M_Sp_df['Price']== '0'].index, inplace=True)

Events_M_Sp_df.columns = Events_M_Sp_df.columns.str.strip().str.lower()

Events_M_Sp_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\188742213.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_M_Sp_df['Date'] = Events_M_Sp_df['Date'].str.replace('8:.05','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\188742213.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_M_Sp_df['Date'] = Events_M_Sp_df['Date'].str.replace('at.05','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\188742213.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_M_Sp_df['Price'] = Events_M_Sp_df['Price'].str.replace('$','')


,name,date,price,links
1,hiit the road berlin by growingannanas + live ...,21.05,39.99,https://www.eventbrite.de/e/hiit-the-road-berl...
2,nature cycling: explore one of the most famous...,27.05,19,https://www.eventbrite.de/e/nature-cycling-exp...
3,nature games in grunewald: let out your inner ...,21.05,15,https://www.eventbrite.de/e/nature-games-in-gr...
4,peacock’s paradise: discover an island full of...,21.05,15,https://www.eventbrite.de/e/peacocks-paradise-...
5,urban foraging: discover the edible treasures ...,21.05,23,https://www.eventbrite.de/e/urban-foraging-dis...
6,fairytale forest: hike through the mystical la...,Tomorrow,11,https://www.eventbrite.de/e/fairytale-forest-h...
7,spreewald canoe tour: discover the unesco bios...,28.05,27,https://www.eventbrite.de/e/spreewald-canoe-to...


#### PAID Events from Eventbrite in Berlin; 
#### Month: June (J); 
#### Category: Sport (Sp).

In [101]:
Events_Names_J_Sp = []
Events_Dates_J_Sp = []
Events_Links_J_Sp = []
Events_Prices_J_Sp = []

for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/sports-and-fitness--events--next-month/paid-events/?page='
    url = url + str(i) 
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_J_Sp.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_J_Sp.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_J_Sp.append(link['href'])

        # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_J_Sp.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    

print(len(Events_Names_J_Sp))
print(len(Events_Dates_J_Sp))
print(len(Events_Links_J_Sp))
print(len(Events_Prices_J_Sp))


5
5
5
5


In [102]:
# Make dictionary, keys that will become dataframe column names
Events_J_dict = {'Name':Events_Names_J_Sp, 'Date':Events_Dates_J_Sp, 'Price':Events_Prices_J_Sp, 'Links':Events_Links_J_Sp}

# Convert dictionary to Pandas dataframe
Events_J_Sp_df = pd.DataFrame(Events_J_dict)
Events_J_Sp_df.index = Events_J_Sp_df.index + 1
Events_J_Sp_df['Date'] = Events_J_Sp_df['Date'].str.replace(',','')
Events_J_Sp_df['Date'] = Events_J_Sp_df['Date'].astype(str)+'.06'
Events_J_Sp_df['Date'] = Events_J_Sp_df['Date'].str.replace('8:.06','Tomorrow')
Events_J_Sp_df['Date'] = Events_J_Sp_df['Date'].str.replace('at.06','Tomorrow')

Events_J_Sp_df.drop(Events_J_Sp_df.loc[Events_J_Sp_df['Price']== '0€'].index, inplace=True)
Events_J_Sp_df['Price'] = Events_J_Sp_df['Price'].str.replace('€','')
Events_J_Sp_df['Price'] = Events_J_Sp_df['Price'].str.replace('$','')
Events_J_Sp_df.drop(Events_J_Sp_df.loc[Events_J_Sp_df['Price']== 'Free'].index, inplace=True)
Events_J_Sp_df.drop(Events_J_Sp_df.loc[Events_J_Sp_df['Price']== 'Kostenlos'].index, inplace=True)
Events_J_Sp_df.drop(Events_J_Sp_df.loc[Events_J_Sp_df['Price']== 'FreiwilligeSpende'].index, inplace=True)
Events_J_Sp_df.drop(Events_J_Sp_df.loc[Events_J_Sp_df['Price']== '0'].index, inplace=True)


Events_J_Sp_df.columns = Events_J_Sp_df.columns.str.strip().str.lower()

Events_J_Sp_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2029261747.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_J_Sp_df['Date'] = Events_J_Sp_df['Date'].str.replace('8:.06','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2029261747.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_J_Sp_df['Date'] = Events_J_Sp_df['Date'].str.replace('at.06','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2029261747.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_J_Sp_df['Price'] = Events_J_Sp_df['Price'].str.replace('$','')


,name,date,price,links
1,peacock’s paradise: discover an island full of...,11.06,15,https://www.eventbrite.de/e/peacocks-paradise-...
2,fairytale forest: hike through the mystical la...,5.06,16,https://www.eventbrite.de/e/fairytale-forest-h...
3,water world: enjoy a day in the gorgeous river...,4.06,15,https://www.eventbrite.de/e/water-world-enjoy-...
4,spreewald canoe tour: discover the unesco bios...,12.06,27,https://www.eventbrite.de/e/spreewald-canoe-to...


#### PAID Events from Eventbrite in Berlin; 
#### Month: July (JL); 
#### Category: Sport (Sp).

In [103]:
# A list with all the Event Names in July

Events_Names_JL_Sp = []
Events_Dates_JL_Sp = []
Events_Links_JL_Sp = []
Events_Prices_JL_Sp = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/sports-and-fitness--events/paid-events/?page='
    url = url + str(i) + '&start_date=2022-07-01&end_date=2022-07-31'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_JL_Sp.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_JL_Sp.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_JL_Sp.append(link['href'])

    # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_JL_Sp.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    
    
print(len(Events_Names_JL_Sp))
print(len(Events_Dates_JL_Sp))
print(len(Events_Links_JL_Sp))
print(len(Events_Prices_JL_Sp))

# NOTEs: NO Events

0
0
0
0


In [104]:
# Make dictionary, keys that will become dataframe column names
#Events_JL_dict = {'Name':Events_Names_JL_Sp, 'Date':Events_Dates_JL_Sp, 'Links':Events_Links_JL_Sp}

# Convert dictionary to Pandas dataframe
#Events_JL_Sp_df = pd.DataFrame(Events_JL_dict)
#Events_JL_Sp_df.index = Events_JL_Sp_df.index + 1
#Events_JL_Sp_df['Date'] = Events_JL_Sp_df['Date'].str.replace('at','Tomorrow')
#Events_JL_Sp_df['Date'] = Events_JL_Sp_df['Date'].str.replace(',','')
#Events_JL_Sp_df

#### PAID Events from Eventbrite in Berlin; 
#### Month: August (A); 
#### Category: Sport (Sp).

In [105]:
# A list with all the Event Names in August

Events_Names_A_Sp = []
Events_Dates_A_Sp = []
Events_Links_A_Sp = []
Events_Prices_A_Sp = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/sports-and-fitness--events/paid-events/?end_date=2022-08-31&page='
    url = url + str(i) + '&start_date=2022-08-01'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_A_Sp.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_A_Sp.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_A_Sp.append(link['href'])

        
         # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_A_Sp.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    

print(len(Events_Names_A_Sp))
print(len(Events_Dates_A_Sp))
print(len(Events_Links_A_Sp))
print(len(Events_Prices_A_Sp))

# NOTEs: NO Events

0
0
0
0


In [106]:
# Make dictionary, keys that will become dataframe column names
#Events_A_dict = {'Name':Events_Names_A_Sp, 'Date':Events_Dates_A_Sp, 'Links':Events_Links_A_Sp}

# Convert dictionary to Pandas dataframe
#Events_A_Sp_df = pd.DataFrame(Events_A_dict)
#Events_A_Sp_df.index = Events_A_Sp_df.index + 1
#Events_A_Sp_df['Date'] = Events_A_Sp_df['Date'].str.replace('at','Tomorrow')
#Events_A_Sp_df['Date'] = Events_A_Sp_df['Date'].str.replace(',','')
#Events_A_Sp_df

#### PAID Events from Eventbrite in Berlin; 
#### Month: May (M); 
#### Category: Outdoor (Out).

In [107]:
# A list with all the Event Names in May

Events_Names_M_Out = []
Events_Dates_M_Out = []
Events_Links_M_Out = []
Events_Prices_M_Out = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/travel-and-outdoor--events--this-month/paid-events/?page='
    url = url + str(i) 
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_M_Out.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_M_Out.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_M_Out.append(link['href'])

 # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_M_Out.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    
    
print(len(Events_Names_M_Out))
print(len(Events_Dates_M_Out))
print(len(Events_Links_M_Out))
print(len(Events_Prices_M_Out))


3
3
3
3


In [108]:
# Make dictionary, keys that will become dataframe column names
Events_M_dict = {'Name':Events_Names_M_Out, 'Date':Events_Dates_M_Out, 'Price': Events_Prices_M_Out, 'Links':Events_Links_M_Out}

# Convert dictionary to Pandas dataframe
Events_M_Out_df = pd.DataFrame(Events_M_dict)
Events_M_Out_df.index = Events_M_Out_df.index + 1
Events_M_Out_df['Date'] = Events_M_Out_df['Date'].str.replace(',','')
Events_M_Out_df['Date'] = Events_M_Out_df['Date'].astype(str)+'.05'
Events_M_Out_df['Date'] = Events_M_Out_df['Date'].str.replace('8:.05','Tomorrow')
Events_M_Out_df['Date'] = Events_M_Out_df['Date'].str.replace('at.05','Tomorrow')

Events_M_Out_df.drop(Events_M_Out_df.loc[Events_M_Out_df['Price']== '0€'].index, inplace=True)
Events_M_Out_df['Price'] = Events_M_Out_df['Price'].str.replace('€','')
Events_M_Out_df['Price'] = Events_M_Out_df['Price'].str.replace('$','')
Events_M_Out_df.drop(Events_M_Out_df.loc[Events_M_Out_df['Price']== 'Free'].index, inplace=True)
Events_M_Out_df.drop(Events_M_Out_df.loc[Events_M_Out_df['Price']== 'Kostenlos'].index, inplace=True)
Events_M_Out_df.drop(Events_M_Out_df.loc[Events_M_Out_df['Price']== 'FreiwilligeSpende'].index, inplace=True)
Events_M_Out_df.drop(Events_M_Out_df.loc[Events_M_Out_df['Price']== '0'].index, inplace=True)

Events_M_Out_df.columns = Events_M_Out_df.columns.str.strip().str.lower()

Events_M_Out_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\1943825868.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_M_Out_df['Date'] = Events_M_Out_df['Date'].str.replace('8:.05','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\1943825868.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_M_Out_df['Date'] = Events_M_Out_df['Date'].str.replace('at.05','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\1943825868.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_M_Out_df['Price'] = Events_M_Out_df['Price'].str.replace('$','')


,name,date,price,links
1,swedish lake: join us on an idyllic hike aroun...,21.05,15,https://www.eventbrite.de/e/swedish-lake-join-...
2,mystic river: venture into haunted forests and...,28.05,14,https://www.eventbrite.de/e/mystic-river-ventu...


#### PAID Events from Eventbrite in Berlin; 
#### Month: June (J); 
#### Category: Outdoor (Out).

In [109]:
# A list with all the Event Names in June

Events_Names_J_Out = []
Events_Dates_J_Out = []
Events_Links_J_Out = []
Events_Preices_J_Out = []


for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/travel-and-outdoor--events--next-month/paid-events/?page='
    url = url + str(i) 
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_J_Out.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_J_Out.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_J_Out.append(link['href'])

        # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Preices_J_Out.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    

print(len(Events_Names_J_Out))
print(len(Events_Dates_J_Out))
print(len(Events_Links_J_Out))
print(len(Events_Preices_J_Out))


3
3
3
3


In [110]:
# Make dictionary, keys that will become dataframe column names
Events_J_dict = {'Name':Events_Names_J_Out, 'Date':Events_Dates_J_Out, 'Price':Events_Preices_J_Out, 'Links':Events_Links_J_Out}

# Convert dictionary to Pandas dataframe
Events_J_Out_df = pd.DataFrame(Events_J_dict)
Events_J_Out_df.index = Events_J_Out_df.index + 1
Events_J_Out_df['Date'] = Events_J_Out_df['Date'].str.replace(',','')
Events_J_Out_df['Date'] = Events_J_Out_df['Date'].astype(str)+'.06'
Events_J_Out_df['Date'] = Events_J_Out_df['Date'].str.replace('8:.06','Tomorrow')
Events_J_Out_df['Date'] = Events_J_Out_df['Date'].str.replace('at.06','Tomorrow')

Events_J_Out_df.drop(Events_J_Out_df.loc[Events_J_Out_df['Price']== '0€'].index, inplace=True)
Events_J_Out_df['Price'] = Events_J_Out_df['Price'].str.replace('€','')
Events_J_Out_df['Price'] = Events_J_Out_df['Price'].str.replace('$','')
Events_J_Out_df.drop(Events_J_Out_df.loc[Events_J_Out_df['Price']== 'Free'].index, inplace=True)
Events_J_Out_df.drop(Events_J_Out_df.loc[Events_J_Out_df['Price']== 'Kostenlos'].index, inplace=True)
Events_J_Out_df.drop(Events_J_Out_df.loc[Events_J_Out_df['Price']== 'FreiwilligeSpende'].index, inplace=True)
Events_J_Out_df.drop(Events_J_Out_df.loc[Events_J_Out_df['Price']== '0'].index, inplace=True)

Events_J_Out_df.columns = Events_J_Out_df.columns.str.strip().str.lower()

Events_J_Out_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\3526546859.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_J_Out_df['Date'] = Events_J_Out_df['Date'].str.replace('8:.06','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\3526546859.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_J_Out_df['Date'] = Events_J_Out_df['Date'].str.replace('at.06','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\3526546859.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_J_Out_df['Price'] = Events_J_Out_df['Price'].str.replace('$','')


,name,date,price,links
1,swedish lake: join us on an idyllic hike aroun...,19.06,15,https://www.eventbrite.de/e/swedish-lake-join-...
2,mystic river: venture into haunted forests and...,12.06,14,https://www.eventbrite.de/e/mystic-river-ventu...


#### PAID Events from Eventbrite in Berlin; 
#### Month: July (JL); 
#### Category: Outdoor (Out).

In [111]:
# A list with all the Event Names in July

Events_Names_JL_Out = []
Events_Dates_JL_Out = []
Events_Links_JL_Out = []
Events_Prices_JL_Out = []

for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/travel-and-outdoor--events/paid-events/?page='
    url = url + str(i) + '&start_date=2022-07-01&end_date=2022-07-31'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    for name in names:
        Events_Names_JL_Out.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_JL_Out.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_JL_Out.append(link['href'])

        
# EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Prices_JL_Out.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    
    
print(len(Events_Names_JL_Out))
print(len(Events_Dates_JL_Out))
print(len(Events_Links_JL_Out))
print(len(Events_Prices_JL_Out))


1
1
1
1


In [112]:
# Make dictionary, keys that will become dataframe column names
Events_JL_dict = {'Name':Events_Names_JL_Out, 'Date':Events_Dates_JL_Out, 'Price': Events_Prices_JL_Out, 'Links':Events_Links_JL_Out}

# Convert dictionary to Pandas dataframe
Events_JL_Out_df = pd.DataFrame(Events_JL_dict)
Events_JL_Out_df.index = Events_JL_Out_df.index + 1
Events_JL_Out_df.drop(Events_JL_Out_df.loc[Events_JL_Out_df['Price']== '0€'].index, inplace=True)
Events_JL_Out_df['Price'] = Events_JL_Out_df['Price'].str.replace('€','')
Events_JL_Out_df['Price'] = Events_JL_Out_df['Price'].str.replace('$','')
Events_JL_Out_df.drop(Events_JL_Out_df.loc[Events_JL_Out_df['Price']== 'Free'].index, inplace=True)
Events_JL_Out_df.drop(Events_JL_Out_df.loc[Events_JL_Out_df['Price']== 'Kostenlos'].index, inplace=True)
Events_JL_Out_df.drop(Events_JL_Out_df.loc[Events_JL_Out_df['Price']== 'FreiwilligeSpende'].index, inplace=True)
Events_JL_Out_df.drop(Events_JL_Out_df.loc[Events_JL_Out_df['Price']== '0'].index, inplace=True)

Events_JL_Out_df.columns = Events_JL_Out_df.columns.str.strip().str.lower()

Events_JL_Out_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\4012814153.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_JL_Out_df['Price'] = Events_JL_Out_df['Price'].str.replace('$','')


,name,date,price,links


#### PAID Events from Eventbrite in Berlin; 
#### Month: August (A); 
#### Category: Outdoor (Out).

In [113]:
# A list with all the Event Names in August

Events_Names_A_Out = []
Events_Dates_A_Out = []
Events_Links_A_Out = []
Events_Price_A_Out = []

for i in range(1,28):
    url = 'https://www.eventbrite.com/d/germany--berlin/travel-and-outdoor--events/paid-events/?end_date=2022-08-31&page='
    url = url + str(i) + '&start_date=2022-08-01'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    names = soup.select('div.eds-is-hidden-accessible')[1::2]
    dates = soup.select('.eds-event-card-content__sub-title')[1::2]
    links = soup.select('.eds-event-card-content__action-link')[1::4]
    
    for name in names:
        Events_Names_A_Out.append(name.get_text(strip=True).lower())
    for date in dates:
        Events_Dates_A_Out.append(date.get_text(strip=True)[9:11].lower())
    for link in links:
        Events_Links_A_Out.append(link['href'])

        
        # EVENTS PRICES
    price_links = soup.select('.eds-event-card-content__primary-content a')[1::2]

    for price in price_links:
        response = requests.get(price['href'])
        soup = BeautifulSoup(response.content, 'html.parser')
        eprice = soup.select('.js-display-price')
    # print(price[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0])
        Events_Price_A_Out.append(eprice[0].get_text(strip=True).replace(u'\xa0', ' ').replace(' ', '').split('–')[0]) 
    
    
    
print(len(Events_Names_A_Out))
print(len(Events_Dates_A_Out))
print(len(Events_Links_A_Out))
print(len(Events_Price_A_Out))


1
1
1
1


In [114]:
# Make dictionary, keys that will become dataframe column names
Events_A_dict = {'Name':Events_Names_A_Out, 'Date':Events_Dates_A_Out, 'Price': Events_Price_A_Out, 'Links':Events_Links_A_Out}

# Convert dictionary to Pandas dataframe
Events_A_Out_df = pd.DataFrame(Events_A_dict)
Events_A_Out_df.index = Events_A_Out_df.index + 1
Events_A_Out_df['Date'] = Events_A_Out_df['Date'].str.replace(',','')
Events_A_Out_df['Date'] = Events_A_Out_df['Date'].astype(str)+'.06'
Events_A_Out_df['Date'] = Events_A_Out_df['Date'].str.replace('8:.06','Tomorrow')
Events_A_Out_df['Date'] = Events_A_Out_df['Date'].str.replace('at.06','Tomorrow')

Events_A_Out_df.drop(Events_A_Out_df.loc[Events_A_Out_df['Price']== '0€'].index, inplace=True)
Events_A_Out_df['Price'] = Events_A_Out_df['Price'].str.replace('€','')
Events_A_Out_df['Price'] = Events_A_Out_df['Price'].str.replace('$','')
Events_A_Out_df.drop(Events_A_Out_df.loc[Events_A_Out_df['Price']== 'Free'].index, inplace=True)
Events_A_Out_df.drop(Events_A_Out_df.loc[Events_A_Out_df['Price']== 'Kostenlos'].index, inplace=True)
Events_A_Out_df.drop(Events_A_Out_df.loc[Events_A_Out_df['Price']== 'FreiwilligeSpende'].index, inplace=True)
Events_A_Out_df.drop(Events_A_Out_df.loc[Events_A_Out_df['Price']== '0'].index, inplace=True)

Events_A_Out_df.columns = Events_A_Out_df.columns.str.strip().str.lower()

Events_A_Out_df

C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2657235424.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_A_Out_df['Date'] = Events_A_Out_df['Date'].str.replace('8:.06','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2657235424.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  Events_A_Out_df['Date'] = Events_A_Out_df['Date'].str.replace('at.06','Tomorrow')
C:\Users\isikh\AppData\Local\Temp\ipykernel_3296\2657235424.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Events_A_Out_df['Price'] = Events_A_Out_df['Price'].str.replace('$','')


,name,date,price,links


## Code to Recommend the Activity

from IPython.display import IFrame


def if_song_in_Top100():
    your_song = input('Hello, what better describe you needs? relax or active? : ')
    
    if str.lower(your_song) in top100_df.values:
        print('Your song is in the top100! :D')
        print('Here a new song for you: ' + random.choice(songs))
        
    else: 
        print('Sorry. Your song is not in the top100. Please, try to listen the following')
        results = sp.search(your_song,limit=1,market="GB")
        # id = results['tracks']['items'][0]['external_urls']['spotify']  # if you return this id you get all the deteail of the user song. 
        
        track_features = sp.audio_features(results['tracks']['items'][0]['id'])  # select the audio features of the user song
        track_features = pd.DataFrame(track_features)
        track_features.drop(['mode','key','type','id','uri','track_href','analysis_url','duration_ms','id','time_signature'], axis = 1, inplace = True) 
        
        X_scaled = scaler.transform(track_features)    # standardize the features of the song as you did for your clusters
        X_scaled_df = pd.DataFrame(X_scaled, columns = track_features.columns)
        
        user_song_cluster = kmeans.predict(X_scaled_df)  # you use the clusters model you saved in the kmeans.pickle to link your song in the proper cluster
        user_song_cluster_int = user_song_cluster[0]
        song_forYou = song_features[song_features['cluster'] == user_song_cluster_int].sample()  # select a random song from the same cluster as the user song
        
        track_id = song_forYou['list_of_id'].to_string(index=False)

        return IFrame(src="https://open.spotify.com/embed/track/"+track_id,    # track the song selected randomly in the cluster
                      width="320", 
                      height="100", 
                      frameborder="0", 
                      allowtransparency="true", 
                      allow="encrypted-media")
         

        
        
if_song_in_Top100()



### All Paid music events

In [115]:
paid_music_events = pd.concat([Events_May_M_df, Events_June_M_df, Events_July_M_df,  Events_Aug_M_df], ignore_index= True)
paid_music_events.index +=1
paid_music_events

,name,date,price,links
1,kiezsalon opening w/ tarta relena and jason sharp,20.05,"11,29",https://www.eventbrite.de/e/kiezsalon-opening-...
2,kiezsalon opening w/ richard skelton and booke...,21.05,"11,29",https://www.eventbrite.de/e/kiezsalon-opening-...
3,banana party,20.05,15,https://www.eventbrite.com/e/banana-party-tick...
4,cdc festival boatride party,28.05,43.80,https://www.eventbrite.de/e/cdc-festival-boatr...
5,"wahn: music, words, dance, performance",27.05,11.83,https://www.eventbrite.de/e/wahn-music-words-d...
...,...,...,...,...
94,halli-galli-party in fürstenberg * open air,27.08,"8,79",https://www.eventbrite.de/e/halli-galli-party-...
95,ü33 1/3 party,6.08,"31,96",https://www.eventbrite.com/e/u33-13-party-tick...
96,dance baby (black meets house),20.08,"31,96",https://www.eventbrite.com/e/dance-baby-black-...
97,dancing with myself | 80er party • factory mag...,27.08,"9,70",https://www.eventbrite.de/e/dancing-with-mysel...


### All paid community events

In [116]:
paid_com_events = pd.concat([Events_M_C_df, Events_J_C_df, Events_JL_C_df,  Events_A_C_df], ignore_index= True)
paid_com_events.index +=1
paid_com_events

,name,date,price,links
1,gypsy tales berlin,28.05,21.43,https://www.eventbrite.co.uk/e/gypsy-tales-ber...
2,silent.move walk'n'dance disco tour // berlin.,20.05,"11,90",https://www.eventbrite.de/e/silentmove-walknda...
3,sunday slips!,22.05,Donation,https://www.eventbrite.de/e/sunday-slips-ticke...
4,iwb may meet-up,25.05,11.83,https://www.eventbrite.de/e/iwb-may-meet-up-ti...
5,pachamama x christa kupfer,Tomorrow,6.51,https://www.eventbrite.com/e/pachamama-x-chris...
6,slam des westens #114 in the dark,25.05,"7,57",https://www.eventbrite.de/e/slam-des-westens-1...
7,noite cabo-verdiana/ stand up comedy com carlo...,28.05,25,https://www.eventbrite.de/e/bilhetes-noite-cab...
8,fundraising im chor | das 1x1 der antragsstellung,21.05,25,https://www.eventbrite.de/e/fundraising-im-cho...
9,"germania, berygtede bunkere og naziarkitektur",28.05,11.83,https://www.eventbrite.de/e/germania-berygtede...
10,dichterasse poetry slam,28.05,"7,57",https://www.eventbrite.de/e/dichterasse-poetry...


### All paid film events

In [117]:
paid_film_events = pd.concat([Events_M_Cin_df, Events_J_Cin_df, Events_JL_Cin_df,  Events_A_Cin_df], ignore_index= True)
paid_film_events.index +=1
paid_film_events

,name,date,price,links
1,english comedy berlin with pizza and shots,20.05,"22,51",https://www.eventbrite.de/e/english-comedy-ber...
2,english stand up comedy - culture shock specia...,21.05,12.54,https://www.eventbrite.co.uk/e/english-stand-u...
3,that's what she said - womxn comedy night,21.05,16,https://www.eventbrite.de/e/thats-what-she-sai...
4,new york comedy club - berlin: main show,23.05,2.14,https://www.eventbrite.com/e/new-york-comedy-c...
5,comedy roulette | the ultimate comedy game show,20.05,13,https://www.eventbrite.de/e/comedy-roulette-th...
6,laughing spree: english comedy on a boat (free...,24.05,2.13,https://www.eventbrite.de/e/laughing-spree-eng...
7,expats in berlin #35 - english comedy show,26.05,8.55,https://www.eventbrite.co.uk/e/expats-in-berli...
8,new york comedy club - berlin: late show - dar...,23.05,2.14,https://www.eventbrite.com/e/new-york-comedy-c...
9,high bars & low morals. english comedy special...,22.05,10,https://www.eventbrite.com/e/high-bars-low-mor...
10,"hook, value and emotions. video content in 202...",25.05,0,https://www.eventbrite.de/e/hook-value-and-emo...


### All paid science events

In [118]:
paid_science_events = pd.concat([Events_M_S_df, Events_J_S_df, Events_JL_S_df,  Events_A_S_df], ignore_index= True)
paid_science_events.index +=1
paid_science_events

,name,date,price,links
1,:clojured berlin conference 2022,11.06,79,https://www.eventbrite.de/e/clojured-berlin-co...
2,openinfra summit berlin 2022,7.06,"1,199",https://www.eventbrite.com/e/openinfra-summit-...
3,gophercon europe 2022 - online and in-person,28.07,126.72,https://www.eventbrite.com/e/gophercon-europe-...
4,4th edition - sx tech eu conference - berlin -...,31.08,74.70,https://www.eventbrite.ie/e/4th-edition-sx-tec...


### All paid sport events

In [119]:
paid_sport_events = pd.concat([Events_M_Sp_df, Events_J_Sp_df], ignore_index= True)
paid_sport_events.index +=1
paid_sport_events

,name,date,price,links
1,hiit the road berlin by growingannanas + live ...,21.05,39.99,https://www.eventbrite.de/e/hiit-the-road-berl...
2,nature cycling: explore one of the most famous...,27.05,19,https://www.eventbrite.de/e/nature-cycling-exp...
3,nature games in grunewald: let out your inner ...,21.05,15,https://www.eventbrite.de/e/nature-games-in-gr...
4,peacock’s paradise: discover an island full of...,21.05,15,https://www.eventbrite.de/e/peacocks-paradise-...
5,urban foraging: discover the edible treasures ...,21.05,23,https://www.eventbrite.de/e/urban-foraging-dis...
6,fairytale forest: hike through the mystical la...,Tomorrow,11,https://www.eventbrite.de/e/fairytale-forest-h...
7,spreewald canoe tour: discover the unesco bios...,28.05,27,https://www.eventbrite.de/e/spreewald-canoe-to...
8,peacock’s paradise: discover an island full of...,11.06,15,https://www.eventbrite.de/e/peacocks-paradise-...
9,fairytale forest: hike through the mystical la...,5.06,16,https://www.eventbrite.de/e/fairytale-forest-h...
10,water world: enjoy a day in the gorgeous river...,4.06,15,https://www.eventbrite.de/e/water-world-enjoy-...


### All paid outdoor events

In [120]:
paid_outdoor_events = pd.concat([Events_M_Out_df, Events_J_Out_df], ignore_index= True)
paid_outdoor_events.index +=1
paid_outdoor_events

,name,date,price,links
1,swedish lake: join us on an idyllic hike aroun...,21.05,15,https://www.eventbrite.de/e/swedish-lake-join-...
2,mystic river: venture into haunted forests and...,28.05,14,https://www.eventbrite.de/e/mystic-river-ventu...
3,swedish lake: join us on an idyllic hike aroun...,19.06,15,https://www.eventbrite.de/e/swedish-lake-join-...
4,mystic river: venture into haunted forests and...,12.06,14,https://www.eventbrite.de/e/mystic-river-ventu...


## Save all the dataframes with pickel

In [121]:
paid_sport_events.to_pickle("paid_sport")
paid_science_events.to_pickle("paid_science")
paid_film_events.to_pickle("paid_film")
paid_com_events.to_pickle("paid_community")
paid_music_events.to_pickle("paid_music")
paid_outdoor_events.to_pickle("paid_travel_out")